# Scala Experiment

We set the Anonymous AWS Credentials Provider as we don't need to authenticate for aws open data


This is a testing notebook to make sure that we the code right first

In [1]:
%%init_spark
launcher.master = "spark://spark-master:7077"
launcher.num_executors = 2
launcher.executor_cores = 8
launcher.executor_memory = '16G'
launcher.packages = ["com.amazon.deequ:deequ:1.2.2-spark-3.0",
                    "org.apache.hadoop:hadoop-aws:3.2.0",
                    "org.locationtech.geomesa:geomesa-spark-jts_2.12:3.2.0"]
launcher.conf.set("spark.hadoop.fs.s3a.aws.credentials.provider", 
                  "org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider")

In [2]:
spark

Intitializing Scala interpreter ...

Spark Web UI available at http://7847ea2e55be:4040
SparkContext available as 'sc' (version = 3.1.1, master = spark://spark-master:7077, app id = app-20210526113541-0004)
SparkSession available as 'spark'


res0: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@6d3cbc40


In [3]:
spark.version

res1: String = 3.1.1


In [4]:
import org.apache.spark.sql.SparkSession
import com.amazon.deequ.VerificationSuite
import com.amazon.deequ.checks.{Check, CheckLevel, CheckStatus}
import com.amazon.deequ.constraints.ConstraintStatus
import java.net.{URI, URLEncoder}
import org.apache.hadoop.fs.{FileSystem, Path, RemoteIterator, LocatedFileStatus}
import scala.collection.mutable.ListBuffer

import org.apache.spark.sql.SparkSession
import com.amazon.deequ.VerificationSuite
import com.amazon.deequ.checks.{Check, CheckLevel, CheckStatus}
import com.amazon.deequ.constraints.ConstraintStatus
import java.net.{URI, URLEncoder}
import org.apache.hadoop.fs.{FileSystem, Path, RemoteIterator, LocatedFileStatus}
import scala.collection.mutable.ListBuffer


# Check Raw Bucket Contents

Checking what data is in the Raw TLC bucket data

In [5]:
val rawS3Dir = "s3a://nyc-tlc/"
//val fileSystem = FileSystem.get(URI.create(rawS3Dir), new Configuration())
val fileSystem = FileSystem.get(URI.create(rawS3Dir), spark.sparkContext.hadoopConfiguration)
val itemlist = fileSystem.listFiles(new Path(rawS3Dir), true)       

rawS3Dir: String = s3a://nyc-tlc/
fileSystem: org.apache.hadoop.fs.FileSystem = S3AFileSystem{uri=s3a://nyc-tlc, workingDir=s3a://nyc-tlc/user/jovyan, inputPolicy=normal, partSize=104857600, enableMultiObjectsDelete=true, maxKeys=5000, readAhead=65536, blockSize=33554432, multiPartThreshold=2147483647, serverSideEncryptionAlgorithm='NONE', blockFactory=org.apache.hadoop.fs.s3a.S3ADataBlocks$DiskBlockFactory@26b4e484, metastore=NullMetadataStore, authoritative=false, useListV1=false, magicCommitter=false, boundedExecutor=BlockingThreadPoolExecutorService{SemaphoredDelegatingExecutor{permitCount=25, available=25, waiting=0}, activeCount=0}, unboundedExecutor=java.util.concurrent.ThreadPoolExecutor@43bf9197[Running, pool size = 0, active threads = 0, queued tasks = 0, completed tasks = 0],...


In [6]:
val limit = 100000
var cnt = 1

limit: Int = 100000
cnt: Int = 1


In [7]:
def loopy(iter: RemoteIterator[LocatedFileStatus]) : ListBuffer[String] = {

    var files = new ListBuffer[String]()
            
    while(cnt <  limit) {
        if (iter.hasNext) {
            val uri = iter.next.getPath.toUri.toString
            files += uri
            cnt += 1
        } else {
            cnt += 1
        }
    }
    return files 
}

loopy: (iter: org.apache.hadoop.fs.RemoteIterator[org.apache.hadoop.fs.LocatedFileStatus])scala.collection.mutable.ListBuffer[String]


In [8]:
val fileList = loopy(itemlist).toList

fileList: List[String] = List(s3a://nyc-tlc/misc/FOIL_2017-12-04.csv, s3a://nyc-tlc/misc/FOIL_fhv_trips_june17.csv, s3a://nyc-tlc/misc/FOIL_medallion_trips_june17.csv, s3a://nyc-tlc/misc/FOIL_weekly_trips_apps.csv, s3a://nyc-tlc/misc/FOIL_weekly_trips_yellow.csv, s3a://nyc-tlc/misc/FOIL_yellow_trips_deadhead.csv, s3a://nyc-tlc/misc/FOIL_yellow_trips_deadhead_apr13.csv, s3a://nyc-tlc/misc/foil26707_1.csv, s3a://nyc-tlc/misc/foil26707_2.csv, s3a://nyc-tlc/misc/foil26707_3.csv, s3a://nyc-tlc/misc/foil26707_4.csv, s3a://nyc-tlc/misc/shared_rides_by_puzone_monthly.csv, s3a://nyc-tlc/misc/taxi%20_zone_lookup.csv, s3a://nyc-tlc/misc/taxi_zones.html, s3a://nyc-tlc/misc/taxi_zones.zip, s3a://nyc-tlc/misc/uber_nyc_data.csv, s3a://nyc-tlc/trip%20data/fhv_tripdata_2015-01.csv, s3a://nyc-tlc/trip%20...


In [16]:
fileList.foreach { println }

s3a://nyc-tlc/misc/FOIL_2017-12-04.csv
s3a://nyc-tlc/misc/FOIL_fhv_trips_june17.csv
s3a://nyc-tlc/misc/FOIL_medallion_trips_june17.csv
s3a://nyc-tlc/misc/FOIL_weekly_trips_apps.csv
s3a://nyc-tlc/misc/FOIL_weekly_trips_yellow.csv
s3a://nyc-tlc/misc/FOIL_yellow_trips_deadhead.csv
s3a://nyc-tlc/misc/FOIL_yellow_trips_deadhead_apr13.csv
s3a://nyc-tlc/misc/foil26707_1.csv
s3a://nyc-tlc/misc/foil26707_2.csv
s3a://nyc-tlc/misc/foil26707_3.csv
s3a://nyc-tlc/misc/foil26707_4.csv
s3a://nyc-tlc/misc/shared_rides_by_puzone_monthly.csv
s3a://nyc-tlc/misc/taxi%20_zone_lookup.csv
s3a://nyc-tlc/misc/taxi_zones.html
s3a://nyc-tlc/misc/taxi_zones.zip
s3a://nyc-tlc/misc/uber_nyc_data.csv
s3a://nyc-tlc/trip%20data/fhv_tripdata_2015-01.csv
s3a://nyc-tlc/trip%20data/fhv_tripdata_2015-02.csv
s3a://nyc-tlc/trip%20data/fhv_tripdata_2015-03.csv
s3a://nyc-tlc/trip%20data/fhv_tripdata_2015-04.csv
s3a://nyc-tlc/trip%20data/fhv_tripdata_2015-05.csv
s3a://nyc-tlc/trip%20data/fhv_tripdata_2015-06.csv
s3a://nyc-tlc/tr

Check the specific trip data

Load the green trip data

In [5]:
val green_trip_data = "s3a://nyc-tlc/trip data/green_tripdata_2018*.csv"
val taxi_test = spark.read.option("header", true).csv(green_trip_data)

green_trip_data: String = s3a://nyc-tlc/trip data/green_tripdata_2018*.csv
taxi_test: org.apache.spark.sql.DataFrame = [VendorID: string, lpep_pickup_datetime: string ... 17 more fields]


In [6]:
taxi_test.printSchema()

root
 |-- VendorID: string (nullable = true)
 |-- lpep_pickup_datetime: string (nullable = true)
 |-- lpep_dropoff_datetime: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- extra: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- tolls_amount: string (nullable = true)
 |-- ehail_fee: string (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- total_amount: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- trip_type: string (nullable = true)



In [11]:
// try deequ test
val verificationResult = VerificationSuite()
    .onData(taxi_test)
    .addCheck(
        Check(CheckLevel.Error, "unit testing my data")
            .isComplete("lpep_pickup_datetime")
            .isNonNegative("fare_amount")
        )
    .run()

verificationResult: com.amazon.deequ.VerificationResult = VerificationResult(Error,Map(Check(Error,unit testing my data,List(CompletenessConstraint(Completeness(lpep_pickup_datetime,None)), ComplianceConstraint(Compliance(fare_amount is non-negative,COALESCE(CAST(fare_amount AS DECIMAL(20,10)), 0.0) >= 0,None)))) -> CheckResult(Check(Error,unit testing my data,List(CompletenessConstraint(Completeness(lpep_pickup_datetime,None)), ComplianceConstraint(Compliance(fare_amount is non-negative,COALESCE(CAST(fare_amount AS DECIMAL(20,10)), 0.0) >= 0,None)))),Error,List(ConstraintResult(CompletenessConstraint(Completeness(lpep_pickup_datetime,None)),Success,None,Some(DoubleMetric(Column,Completeness,lpep_pickup_datetime,Success(1.0)))), ConstraintResult(ComplianceConstraint(Compliance(fare_amou...


In [12]:
if (verificationResult.status == CheckStatus.Success) {
  println("The data passed the test, everything is fine!")
} else {
  println("We found errors in the data:\n")

  val resultsForAllConstraints = verificationResult.checkResults
    .flatMap { case (_, checkResult) => checkResult.constraintResults }

  resultsForAllConstraints
    .filter { _.status != ConstraintStatus.Success }
    .foreach { result => println(s"${result.constraint}: ${result.message.get}") }
}

We found errors in the data:

ComplianceConstraint(Compliance(fare_amount is non-negative,COALESCE(CAST(fare_amount AS DECIMAL(20,10)), 0.0) >= 0,None)): Value: 0.9973816047886623 does not meet the constraint requirement!


## Part 2 - Run some ETL

In [7]:
import org.apache.spark.sql.functions.{to_timestamp, year, month, dayofmonth, col, hour, to_date}

import org.apache.spark.sql.functions.{to_timestamp, year, month, dayofmonth, col, hour, to_date}


In [8]:
taxi_test.printSchema()

root
 |-- VendorID: string (nullable = true)
 |-- lpep_pickup_datetime: string (nullable = true)
 |-- lpep_dropoff_datetime: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- extra: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- tolls_amount: string (nullable = true)
 |-- ehail_fee: string (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- total_amount: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- trip_type: string (nullable = true)



In [9]:
taxi_test.select("PULocationID").distinct().collect() //.take(30)

res4: Array[org.apache.spark.sql.Row] = Array([125], [7], [51], [124], [205], [169], [54], [15], [234], [232], [155], [154], [132], [200], [11], [101], [138], [69], [29], [42], [112], [73], [87], [64], [3], [113], [30], [34], [133], [162], [59], [146], [250], [139], [160], [8], [258], [203], [22], [28], [184], [85], [52], [16], [35], [251], [171], [183], [187], [71], [188], [98], [223], [47], [195], [99], [107], [214], [179], [248], [202], [96], [221], [43], [5], [31], [163], [100], [18], [70], [174], [206], [168], [224], [61], [218], [27], [75], [166], [219], [17], [126], [131], [140], [26], [227], [120], [46], [130], [147], [164], [207], [78], [208], [77], [228], [89], [136], [198], [257], [118], [6], [185], [256], [230], [201], [177], [244], [60], [68], [246], [229], [90], [194], [19...


In [10]:
// HH:mm:ss
val formatted = taxi_test
    .withColumn("pickupDate", to_timestamp(col("lpep_pickup_datetime"), "yyyy-MM-dd HH:mm:SS"))
    .withColumn("dropOffDate", to_timestamp(col("lpep_dropoff_datetime"), "yyyy-MM-dd HH:mm:SS"))
    .withColumn("tripDistance", $"trip_distance".cast("float"))
    .drop("lpep_pickup_datetime")
    .drop("lpep_dropoff_datetime")

formatted: org.apache.spark.sql.DataFrame = [VendorID: string, store_and_fwd_flag: string ... 18 more fields]


In [11]:
formatted.createTempView("formatted_taxi_data")

In [12]:
// calculate the average cost between different areas
val analysis_df = spark.sql("""SELECT PULocationID, DOLocationID, avg(tripDistance), avg(fare_amount) 
            FROM formatted_taxi_data 
            GROUP BY PULocationID, DOLocationID""")

analysis_df: org.apache.spark.sql.DataFrame = [PULocationID: string, DOLocationID: string ... 2 more fields]


In [13]:
analysis_df.collect()

res6: Array[org.apache.spark.sql.Row] = Array([33,112,4.946209033487564,18.089982285208144], [49,17,1.1625126630102134,6.842870299309287], [83,129,1.303071470070813,7.878322504430006], [80,33,4.309654428685973,16.351879049676025], [112,97,4.097139615612051,15.866742338251987], [255,62,4.4193831818126075,17.678504672897198], [217,217,0.6965442420790385,6.5828964941569295], [41,31,8.199055146044634,26.00228346456693], [112,25,5.418259954052651,18.96981132075472], [40,244,14.994736771834525,44.60526315789474], [145,181,7.8906250157777,27.201654411764707], [182,20,3.253474026531368,13.570064935064934], [33,140,7.692936797124302,25.73375464684015], [123,140,16.778518535472728,51.97925925925926], [253,138,2.8031578722753023,12.223684210526315], [42,231,10.189582743747629,35.29907913669065], [...


# Advanced Analysis

Trying Some GeoMesa Experiments - first load a dataset with long lats

In [5]:
val green_trip_data_2015_h1 = "s3a://nyc-tlc/trip data/green_tripdata_2015-0[1-6].csv"
val green_trip_2015_h1 = spark.read.option("header", true).csv(green_trip_data_2015_h1)

green_trip_data_2015_h1: String = s3a://nyc-tlc/trip data/green_tripdata_2015-0[1-6].csv
green_trip_2015_h1: org.apache.spark.sql.DataFrame = [VendorID: string, lpep_pickup_datetime: string ... 19 more fields]


In [6]:
// load geo mesa bits
import org.locationtech.jts.geom._
import org.locationtech.geomesa.spark.jts._

import org.locationtech.jts.geom._
import org.locationtech.geomesa.spark.jts._


In [14]:
spark.withJTS

res5: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@6d3cbc40


In [10]:
import org.apache.spark.sql.types._

import org.apache.spark.sql.types._


In [7]:
green_trip_2015_h1.printSchema()

root
 |-- VendorID: string (nullable = true)
 |-- lpep_pickup_datetime: string (nullable = true)
 |-- Lpep_dropoff_datetime: string (nullable = true)
 |-- Store_and_fwd_flag: string (nullable = true)
 |-- RateCodeID: string (nullable = true)
 |-- Pickup_longitude: string (nullable = true)
 |-- Pickup_latitude: string (nullable = true)
 |-- Dropoff_longitude: string (nullable = true)
 |-- Dropoff_latitude: string (nullable = true)
 |-- Passenger_count: string (nullable = true)
 |-- Trip_distance: string (nullable = true)
 |-- Fare_amount: string (nullable = true)
 |-- Extra: string (nullable = true)
 |-- MTA_tax: string (nullable = true)
 |-- Tip_amount: string (nullable = true)
 |-- Tolls_amount: string (nullable = true)
 |-- Ehail_fee: string (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- Total_amount: string (nullable = true)
 |-- Payment_type: string (nullable = true)
 |-- Trip_type : string (nullable = true)



In [15]:
val test_process = green_trip_2015_h1
    .withColumn("pickup_longitude", col("Pickup_longitude").cast(DoubleType))
    .withColumn("pickup_latitude", col("Pickup_latitude").cast(DoubleType))
    .withColumn("pickup_point", st_makePoint(col("pickup_longitude"), col("pickup_latitude")))

test_process: org.apache.spark.sql.DataFrame = [VendorID: string, lpep_pickup_datetime: string ... 20 more fields]


In [16]:
test_process.take(2)

java.lang.IllegalStateException:  Cannot call methods on a stopped SparkContext.

In [9]:
// stop the session
spark.stop()